In [ ]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.models import Model, load_model
from keras.layers import Conv2D, MaxPool2D, Input, concatenate, UpSampling2D, Dropout
from keras.layers import Conv2DTranspose, AvgPool2D, GaussianNoise, BatchNormalization
import tensorflow as tf
import keras.backend as k
from tensorflow.keras.optimizers import Adam
from keras.losses import binary_crossentropy
from keras.metrics import MeanIoU
import keras
from keras.callbacks import TensorBoard, ModelCheckpoint

In [ ]:
def IoU(y_true, y_pred, eps=1e-6):
    intersection = k.sum(y_true * y_pred, axis=[1,2,3])
    union = k.sum(y_true, axis=[1,2,3]) + k.sum(y_pred, axis=[1,2,3]) - intersection
    return -k.mean((intersection) / (union + eps), axis=0)

In [ ]:
def get_model():
    k.clear_session()
    inputs = Input((256,256,3))
    s = BatchNormalization()(inputs)
    s = Dropout(0.5)(s)
    
    c1 = Conv2D(8, (3, 3), activation="relu", padding="same")(s)
    c1 = Conv2D(8, (3, 3), activation="relu", padding="same")(c1)
    p1 = MaxPool2D((2, 2))(c1)

    c2 = Conv2D(16, (3, 3), activation="relu", padding="same")(p1)
    c2 = Conv2D(16, (3, 3), activation="relu", padding="same")(c2)
    p2 = MaxPool2D((2, 2))(c2)

    c3 = Conv2D(32, (3, 3), activation="relu", padding="same")(p2)
    c3 = Conv2D(32, (3, 3), activation="relu", padding="same")(c3)
    p3 = MaxPool2D((2, 2))(c3)

    c4 = Conv2D(64, (3, 3), activation="relu", padding="same")(p3)
    c4 = Conv2D(64, (3, 3), activation="relu", padding="same")(c4)
    p4 = MaxPool2D((2, 2))(c4)

    c5 = Conv2D(128, (3, 3), activation="relu", padding="same")(p4)
    c5 = Conv2D(128, (3, 3), activation="relu", padding="same")(c5)

    u6 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding="same")(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(64, (3, 3), activation="relu", padding="same")(u6)
    c6 = Conv2D(64, (3, 3), activation="relu", padding="same")(c6)

    u7 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding="same")(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(32, (3, 3), activation="relu", padding="same")(u7)
    c7 = Conv2D(32, (3, 3), activation="relu", padding="same")(c7)

    u8 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding="same")(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(16, (3, 3), activation="relu", padding="same")(u8)
    c8 = Conv2D(16, (3, 3), activation="relu", padding="same")(c8)

    u9 = Conv2DTranspose(8, (2, 2), strides=(2, 2), padding="same")(c8)
    u9 = concatenate([u9, c1])
    c9 = Conv2D(8, (3, 3), activation="relu", padding="same")(u9)
    c9 = Conv2D(8, (3, 3), activation="relu", padding="same")(c9)

    outputs = Conv2D(1, (1, 1), activation="sigmoid")(c9)

    model = Model(inputs=inputs, outputs=outputs)
    print(model.summary())
    model.compile(optimizer=Adam(lr=1e-4), loss=IoU, metrics=["binary_accuracy"])
    return model


In [ ]:
X = np.load('/content/drive/MyDrive/All-datasets/membrane/Data.ImageSegnentation/train_image.npy')
Y = np.load('/content/drive/MyDrive/All-datasets/membrane/Data.ImageSegnentation/mask_image.npy')

In [ ]:
def t_generator(X_train, Y_train, batch_size):
    features = k.zeros(shape=(batch_size, 256,256,3))
    labels = k.zeros(shape=(batch_size, 256, 256, 1))
    while True:
        start = 0
        end = batch_size
        for i in range(243):
            features = X_train[start:end]
            labels = Y_train[start:end]
            start = end
            end = end + batch_size
            yield features / 255.0, labels / 255.0
            
def v_generator(X_val, Y_val, batch_size):
    features = k.zeros(shape=(batch_size, 256,256,3))
    labels = k.zeros(shape=(batch_size, 256,256,1))
    while True:
        start = 0
        end = batch_size
        for i in range(27):
            features = X_val[start:end]
            labels = Y_val[start:end]
            start = end
            end = end + batch_size
            yield features / 255.0, labels / 255.0
            
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,
                                                   shuffle=True,
                                                   random_state=256,
                                                   test_size=0.1)

del X, Y
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train,
                                                 shuffle=True,
                                                 random_state=256,
                                                 test_size=0.1)

model = get_model()
model_chkpt = ModelCheckpoint("unet2_model_best.h5",
                             monitor="binary_accuracy", verbose=1,
                             save_best_only=True)

history = model.fit(t_generator(X_train, Y_train, 32),
                   steps_per_epoch=X_train.shape[0] // 32,
                   epochs=500,callbacks=[model_chkpt],
                   validation_data=v_generator(X_val, Y_val, 32),
                   validation_steps=X_val.shape[0] // 32)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 256, 3)  12          input_1[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 256, 256, 3)  0           batch_normalization[0][0]        
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 8)  224         dropout[0][0]                    
______________________________________________________________________________________________

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/500
25/25 [==============================] - 228s 9s/step - loss: -0.1335 - binary_accuracy: 0.8143 - val_loss: -0.1498 - val_binary_accuracy: 0.5186

Epoch 00001: binary_accuracy improved from -inf to 0.81428, saving model to unet2_model_best.h5
Epoch 2/500
25/25 [==============================] - 6s 11ms/step - loss: nan - binary_accuracy: 0.7966 - val_loss: 0.0000e+00 - val_binary_accuracy: 0.0000e+00

Epoch 00002: binary_accuracy did not improve from 0.81428
Epoch 3/500
25/25 [==============================] - 0s 10ms/step - loss: 0.0000e+00 - binary_accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_binary_accuracy: 0.0000e+00

Epoch 00003: binary_accuracy did not improve from 0.81428
Epoch 4/500
25/25 [==============================] - 0s 11ms/step - loss: 0.0000e+00 - binary_accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_binary_accuracy: 0.0000e+00

Epoch 00004: binary_accuracy did not improve from 0.81428
Epoch 5/500
25/25 [==============================] - 0s 11ms/st

In [ ]:
X = np.load("/content/drive/MyDrive/All-datasets/membrane/Data.ImageSegnentation/train_image.npy")
Y = np.load("/content/drive/MyDrive/All-datasets/membrane/Data.ImageSegnentation/mask_image.npy")

In [ ]:
def t_generator(X_train, Y_train, batch_size):
    features = k.zeros(shape=(batch_size, 256,256,3))
    labels = k.zeros(shape=(batch_size, 256, 256, 1))
    while True:
        start = 0
        end = batch_size
        for i in range(243):
            features = X_train[start:end]
            labels = Y_train[start:end]
            start = end
            end = end + batch_size
            yield features / 255.0, labels / 255.0
            
def v_generator(X_val, Y_val, batch_size):
    features = k.zeros(shape=(batch_size, 256,256,3))
    labels = k.zeros(shape=(batch_size, 256,256,1))
    while True:
        start = 0
        end = batch_size
        for i in range(27):
            features = X_val[start:end]
            labels = Y_val[start:end]
            start = end
            end = end + batch_size
            yield features / 255.0, labels / 255.0
            
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,
                                                   shuffle=True,
                                                   random_state=256,
                                                   test_size=0.1)

del X, Y
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train,
                                                 shuffle=True,
                                                 random_state=256,
                                                 test_size=0.1)

model = get_model()
model_chkpt = ModelCheckpoint("unet2_model_best.h5",
                             monitor="binary_accuracy", verbose=1,
                             save_best_only=True)

history = model.fit(t_generator(X_train, Y_train, 32),
                   steps_per_epoch=X_train.shape[0] // 32,
                   epochs=500,callbacks=[model_chkpt],
                   validation_data=v_generator(X_val, Y_val, 32),
                   validation_steps=X_val.shape[0] // 32)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 256, 3)  12          input_1[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 256, 256, 3)  0           batch_normalization[0][0]        
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 8)  224         dropout[0][0]                    
______________________________________________________________________________________________

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


25/25 [==============================] - 232s 9s/step - loss: -0.1490 - binary_accuracy: 0.5546 - val_loss: -0.1534 - val_binary_accuracy: 0.1934

Epoch 00001: binary_accuracy improved from -inf to 0.55457, saving model to unet2_model_best.h5
Epoch 2/500
25/25 [==============================] - 6s 11ms/step - loss: nan - binary_accuracy: 0.4701 - val_loss: 0.0000e+00 - val_binary_accuracy: 0.0000e+00

Epoch 00002: binary_accuracy did not improve from 0.55457
Epoch 3/500
25/25 [==============================] - 0s 11ms/step - loss: 0.0000e+00 - binary_accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_binary_accuracy: 0.0000e+00

Epoch 00003: binary_accuracy did not improve from 0.55457
Epoch 4/500
25/25 [==============================] - 0s 12ms/step - loss: 0.0000e+00 - binary_accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_binary_accuracy: 0.0000e+00

Epoch 00004: binary_accuracy did not improve from 0.55457
Epoch 5/500
25/25 [==============================] - 0s 12ms/step - loss: 0